In [1]:
%pip -q install google-genai

# Instalar Framework de agentes do Google ################################################
%pip -q install google-adk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [3]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [4]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [5]:
from IPython.display import HTML, Markdown

In [6]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [7]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [8]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [9]:
############################################################################################################################
# --- Agente 1: Coleta, Organiza os Dados e Interage com o Gemini API, Recebe a estrutura geral do plano de exercícios --- #
############################################################################################################################
def agente_planoGeralExerc(dados_usuario):

    buscador = Agent(
        name="agente_planoGeralExerc",
        model="gemini-2.0-flash",
        instruction="""
        Você é um especialista em criar planos de exercícios gerais com base nas informações fornecidas pelo usuário.
        Seu objetivo é analisar os dados do usuário abaixo e gerar uma estrutura semanal de treinamento que inclua:
        - Uma sugestão de divisão dos dias de treino (por exemplo, focando em grupos musculares específicos ou utilizando um sistema como treino A/B/C).
        - Os principais grupos musculares a serem trabalhados em cada dia de treino sugerido.
        - A frequência semanal recomendada para o trabalho de cada grupo muscular, levando em consideração o objetivo e o nível de atividade do usuário.
        - A divisão em dias ou períodos dentro do mesmo treino para prática de exercícios aeróbicos e exercícios de resistência.

        Você deve levar em consideração as seguintes informações do usuário:
        - Idade.
        - Objetivo principal (hipertrofia, emagrecimento, força, etc.).
        - Nível de atividade física atual.
        - Histórico de exercícios.
        - Restrições físicas ou condições médicas (se houver).
        - Equipamento disponível.
        - Tempo disponível por sessão de treino.
        - Frequência semanal de treino desejada.
        - Preferências de treino (se relevantes para a estrutura geral).

        Sua saída deve ser concisa e clara, fornecendo a estrutura geral do plano para que outro agente possa detalhar os exercícios específicos.
        """,
        description="Agente que Constroi o Plano Geral de Exercícios"
    )

    entrada_agente_planoGeralExerc = f"""
    Informações do usuário:

    Idade: {dados_usuario.get('idade', 'não especificada')} anos.
    Objetivo principal: {dados_usuario.get('objetivo', 'não especificada')}.
    Nível de atividade física atual: {dados_usuario.get('nivel_atividade', 'não especificada')}.
    Histórico de exercícios: {dados_usuario.get('historico_exercicios', 'não especificada')}.
    Restrições físicas ou condições médicas: {dados_usuario.get('restricoes', 'nenhuma')}.
    Equipamento disponível: {dados_usuario.get('equipamento', 'nenhum')}.
    Tempo disponível por sessão de treino: {dados_usuario.get('tempo_por_treino', 'não especificada')} minutos.
    Frequência semanal de treino desejada: {dados_usuario.get('frequencia_semanal', 'não especificada')} dias.
    Preferências de treino: {dados_usuario.get('preferencias', 'nenhuma')}.

    Gere a estrutura geral do plano de exercícios semanal com base nessas informações.
    """

    plano_geral = call_agent(buscador, entrada_agente_planoGeralExerc)
    return plano_geral

In [20]:
#########################################################################################################
# --- Agente 2: Detalha o Plano de Exercícios e verifica se está de acordo com as Diretrizes da OMS --- #
#########################################################################################################
def agente_planoFinalExerc(dados_usuario, plano_geral):
    revisor = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Revisor #################################################
        instruction=f"""
        Você é um especialista em criar planos de exercícios detalhados com base em uma estrutura geral e nas informações do usuário, seguindo as diretrizes da Organização Mundial da Saúde (OMS).
        Seu objetivo é pegar a estrutura geral do plano de exercícios e as informações detalhadas do usuário para gerar um plano semanal completo, incluindo:

        Formate o plano de exercícios da seguinte maneira:

        1.  **Informações Gerais:**
            - "**Objetivo:**" [Objetivo do Usuário]
            - "**Frequência:**" [Frequência Semanal]

        2.  **Estrutura Semanal:** Para cada dia de treino:
            - Para cada dia, comece com um título em negrito indicando o dia e o foco (ex: "**Dia 1: [Foco do Treino]**", "**Dia 2: [Foco do Treino]**", etc).
            - Para cada dia, abaixo do título, liste os exercícios em uma tabela. Para cada exercício, inclua as informações nas seguintes colunas:
                - "**Exercício:**" [Nome do Exercício]
                - "**Tipo:**" [Classificação: Aeróbico ou Fortalecimento]
                - "**Duração Estimada:**" [Duração em minutos (por série ou total do exercício)]
                - "**Séries:**" [Número de Séries]
                - "**Repetições:**" [Número de Repetições]
                - "**Descanso:**" [Descanso em segundos]
                - "**Notas:**" [Notas adicionais, se houver]

         3.  **Aquecimento Sugerido:** (Lista de sugestões)

         4.  **Resfriamento Sugerido:** (Lista de sugestões)

         5.  **Recomendação da OMS:** Com base na idade ([Idade do Usuário]), nível de atividade física atual ([Nível de Atividade]), e objetivo ([Objetivo do Usuário]), determine a recomendação geral da OMS para atividade física (quantidade, intensidade, tipos). Formule uma frase clara com essa recomendação.

         6.  **Comparativo com o Plano:** Compare o plano de exercícios montado com a recomendação da OMS para o usuário. Formule uma frase indicando se o plano está alinhado com as diretrizes (e em quais aspectos) ou se há alguma diferença.

         """,
        description="Agente que Detalha o Plano de Exercícios e verifica se está de acordo com as Diretrizes da OMS",
        tools=[google_search]
       )

    entrada_do_agente_planoFinalExerc = f"""
      Estrutura geral do plano de exercícios:{plano_geral}
      Informações detalhadas do usuário:
      Idade: {dados_usuario.get('idade', 'não especificada')} anos.
      Objetivo principal: {dados_usuario.get('objetivo', 'não especificada')}.
      Nível de atividade física atual: {dados_usuario.get('nivel_atividade', 'não especificada')}.
      Histórico de exercícios: {dados_usuario.get('historico_exercicios', 'não especificada')}.
      Restrições físicas ou condições médicas: {dados_usuario.get('restricoes', 'nenhuma')}.
      Equipamento disponível: {dados_usuario.get('equipamento', 'nenhum')}.
      Tempo disponível por sessão de treino: {dados_usuario.get('tempo_por_treino', 'não especificada')} minutos.
      Frequência semanal de treino desejada: {dados_usuario.get('frequencia_semanal', 'não especificada')} dias.
      Preferências de treino: {dados_usuario.get('preferencias', 'nenhuma')}.

      Com base nessa estrutura e nas informações do usuário, crie um plano de exercícios semanal completo e detalhado.
      Certifique-se de seguir as diretrizes da OMS para a prática de exercícios físicos ao elaborar o plano.
     """
    # Executa o agente

    plano_final = call_agent(revisor,  entrada_do_agente_planoFinalExerc)

    return plano_final

In [49]:
def coletar_dados_usuario():
    dados_usuario = {}

    print("Olá! Para criarmos seu plano de exercícios personalizado, preciso de algumas informações.")

    dados_usuario['nome'] = input("Qual o seu primeiro nome? ")
    dados_usuario['idade'] = input("Qual a sua idade? ")
    dados_usuario['objetivo'] = input("Qual seu principal objetivo (hipertrofia, emagrecimento, etc.)? ")
    dados_usuario['nivel_atividade'] = input("Qual seu nível de atividade física atual (sedentário, leve, moderado, intenso)? ")
    dados_usuario['historico_exercicios'] = input("Você já praticou exercícios antes? Se sim, por quanto tempo e quais tipos? ")
    dados_usuario['restricoes'] = input("Você possui alguma restrição física ou condição médica que eu deva saber? (Se não, digite 'não') ")
    dados_usuario['equipamento'] = input("Qual equipamento você tem disponível para se exercitar? (Se nenhum, digite 'nenhum') ")
    dados_usuario['tempo_por_treino'] = input("Quanto tempo você tem disponível para cada sessão de treino (em minutos)? ")
    dados_usuario['frequencia_semanal'] = input("Quantos dias por semana você gostaria de treinar? ")
    dados_usuario['preferencias'] = input("Você tem alguma preferência por tipos de exercícios ou formatos de treino? ")

    # Você pode adicionar mais perguntas conforme necessário

    return dados_usuario

In [50]:
def main():
    print("# 💪 Bem-vindo ao seu criador de planos de treinamento personalizado! 🏃‍♀️ #")
    print("\n⚠️ **Importante:** ⚠️ Antes de iniciar qualquer programa de exercícios, é fundamental consultar um médico ou profissional de educação física \npara avaliar sua condição física e garantir que o plano seja adequado para você. \n🩺Este chatbot oferece sugestões baseadas em informações fornecidas, mas não substitui a orientação profissional. \n")

    dados_do_usuario = coletar_dados_usuario()

    plano_geral = agente_planoGeralExerc(dados_do_usuario)

    if plano_geral:
        print("\n 🧠 Gerando seu plano de treinamento detalhado...\n")
        plano_final = agente_planoFinalExerc(dados_do_usuario, plano_geral)

        if plano_final:
            print(f"\n🎉 **{dados_do_usuario['nome']}**, seu plano de treinamento personalizado está pronto! 🎉\n ")
            display(to_markdown(plano_final))

        else:
            print("\n❌ Houve um erro ao gerar o plano de treinamento final. Por favor, tente novamente.\n")
    else:
        print("\n❌ Houve um erro ao gerar a estrutura geral do plano. Por favor, tente novamente.\n")

if __name__ == "__main__":
    main()


# 💪 Bem-vindo ao seu criador de planos de treinamento personalizado! 🏃‍♀️ #

⚠️ **Importante:** ⚠️ Antes de iniciar qualquer programa de exercícios, é fundamental consultar um médico ou profissional de educação física 
para avaliar sua condição física e garantir que o plano seja adequado para você. 
🩺Este chatbot oferece sugestões baseadas em informações fornecidas, mas não substitui a orientação profissional. 

Olá! Para criarmos seu plano de exercícios personalizado, preciso de algumas informações.
Qual o seu primeiro nome? Liana
Qual a sua idade? 42
Qual seu principal objetivo (hipertrofia, emagrecimento, etc.)? hipertrofia
Qual seu nível de atividade física atual (sedentário, leve, moderado, intenso)? leve
Você já praticou exercícios antes? Se sim, por quanto tempo e quais tipos? sim. corrida
Você possui alguma restrição física ou condição médica que eu deva saber? (Se não, digite 'não') não
Qual equipamento você tem disponível para se exercitar? (Se nenhum, digite 'nenhum') academ

> Com certeza! Aqui está o plano de exercícios detalhado, seguindo a estrutura fornecida e as diretrizes da OMS, com base nas suas informações:
> 
> **1. Informações Gerais:**
> 
> *   **Objetivo:** Hipertrofia
> *   **Frequência:** 5 dias por semana
> 
> **2. Estrutura Semanal:**
> 
> *   **Dia 1: Membros Inferiores (A)**
> 
> | Exercício                  | Tipo         | Duração Estimada | Séries | Repetições | Descanso | Notas                                    |
> | :------------------------- | :----------- | :--------------- | :----- | :--------- | :------- | :--------------------------------------- |
> | Agachamento Livre          | Fortalecimento | 2 minutos        | 3      | 8-12       | 90s      | Priorizar a técnica correta.             |
> | Leg Press 45°              | Fortalecimento | 2 minutos        | 3      | 10-15      | 90s      | Ajustar a carga para a faixa de repetição. |
> | Extensora                  | Fortalecimento | 1.5 minutos      | 3      | 12-15      | 60s      | Controlar a fase excêntrica.             |
> | Flexora Deitado            | Fortalecimento | 1.5 minutos      | 3      | 12-15      | 60s      | Manter a curvatura natural da coluna.    |
> | Elevação de Gêmeos no Smith | Fortalecimento | 1 minuto         | 4      | 15-20      | 45s      | Amplitude completa.                      |
> 
> *   **Dia 2: Superiores - Empurrar (B)**
> 
> | Exercício                | Tipo         | Duração Estimada | Séries | Repetições | Descanso | Notas                                                                     |
> | :----------------------- | :----------- | :--------------- | :----- | :--------- | :------- | :------------------------------------------------------------------------ |
> | Supino Reto com Barra    | Fortalecimento | 2 minutos        | 3      | 8-12       | 90s      | Manter a escápula estabilizada.                                           |
> | Supino Inclinado Halteres | Fortalecimento | 2 minutos        | 3      | 10-15      | 90s      | Amplitude completa.                                                       |
> | Desenvolvimento Halteres   | Fortalecimento | 1.5 minutos      | 3      | 8-12       | 60s      | Controlar a descida.                                                      |
> | Elevação Lateral         | Fortalecimento | 1 minuto         | 3      | 12-15      | 45s      | Priorizar a técnica sobre a carga.                                       |
> | Tríceps Testa com Barra  | Fortalecimento | 1.5 minutos      | 3      | 10-15      | 60s      | Cotovelos apontando para o teto.                                         |
> 
> *   **Dia 3: Superiores - Puxar (C)**
> 
> | Exercício                      | Tipo         | Duração Estimada | Séries | Repetições | Descanso | Notas                                               |
> | :----------------------------- | :----------- | :--------------- | :----- | :--------- | :------- | :-------------------------------------------------- |
> | Barra Fixa (com assistência)   | Fortalecimento | 2 minutos        | 3      | 6-10       | 90s      | Priorizar a amplitude e a técnica.                  |
> | Remada Curvada com Barra       | Fortalecimento | 2 minutos        | 3      | 8-12       | 90s      | Manter a curvatura natural da coluna.               |
> | Puxada Alta (Pulldown)         | Fortalecimento | 1.5 minutos      | 3      | 10-15      | 60s      | Controlar a fase excêntrica.                        |
> | Remada Serrote com Halter      | Fortalecimento | 1.5 minutos      | 3      | 10-15      | 60s      | Manter a estabilidade do tronco.                    |
> | Rosca Direta com Barra         | Fortalecimento | 1.5 minutos      | 3      | 8-12       | 60s      | Controlar a descida.                                |
> | Encolhimento com Halteres      | Fortalecimento | 1 minuto         | 3      | 15-20      | 45s      | Amplitude completa, focar na contração do trapézio. |
> 
> *   **Dia 4: Membros Inferiores (A)**
> 
> | Exercício            | Tipo         | Duração Estimada | Séries | Repetições | Descanso | Notas                                    |
> | :------------------- | :----------- | :--------------- | :----- | :--------- | :------- | :--------------------------------------- |
> | Agachamento Frontal  | Fortalecimento | 2 minutos        | 3      | 8-12       | 90s      | Manter a postura ereta.                  |
> | Hack Squat           | Fortalecimento | 2 minutos        | 3      | 10-15      | 90s      | Ajustar a carga para a faixa de repetição. |
> | Cadeira Adutora      | Fortalecimento | 1.5 minutos      | 3      | 12-15      | 60s      | Controlar o movimento.                   |
> | Stiff                | Fortalecimento | 1.5 minutos      | 3      | 10-15      | 60s      | Manter a curvatura natural da coluna.    |
> | Elevação de Gêmeos Sentado | Fortalecimento | 1 minuto         | 4      | 15-20      | 45s      | Amplitude completa.                      |
> 
> *   **Dia 5: Superiores - Empurrar (B)**
> 
> | Exercício                      | Tipo         | Duração Estimada | Séries | Repetições | Descanso | Notas                                                                     |
> | :----------------------------- | :----------- | :--------------- | :----- | :--------- | :------- | :------------------------------------------------------------------------ |
> | Supino Inclinado com Barra      | Fortalecimento | 2 minutos        | 3      | 8-12       | 90s      | Manter a escápula estabilizada.                                           |
> | Crucifixo Reto com Halteres     | Fortalecimento | 2 minutos        | 3      | 10-15      | 90s      | Amplitude completa.                                                       |
> | Elevação Frontal com Halteres   | Fortalecimento | 1.5 minutos      | 3      | 10-15      | 60s      | Controlar a descida.                                                      |
> | Desenvolvimento Arnold         | Fortalecimento | 1.5 minutos      | 3      | 8-12       | 60s      | Priorizar a técnica sobre a carga.                                       |
> | Tríceps Francês com Halteres   | Fortalecimento | 1.5 minutos      | 3      | 10-15      | 60s      | Cotovelos apontando para o teto.                                         |
> 
> **3. Aquecimento Sugerido:**
> 
> *   5 minutos de bicicleta ergométrica ou esteira (leve)
> *   Mobilidade articular (rotações de braços, pernas, tronco)
> *   2 séries de 15 repetições de agachamento com o peso do corpo
> 
> **4. Resfriamento Sugerido:**
> 
> *   5-10 minutos de caminhada leve na esteira
> *   Alongamentos estáticos dos principais grupos musculares trabalhados (30 segundos por alongamento)
> 
> **5. Recomendação da OMS:**
> 
> Considerando sua idade (42 anos) e nível de atividade física atual (leve), a OMS recomenda pelo menos 150 minutos de atividade física aeróbica de intensidade moderada por semana, ou 75 minutos de atividade física aeróbica de intensidade vigorosa, ou uma combinação equivalente. Além disso, é recomendado realizar atividades de fortalecimento muscular que envolvam todos os principais grupos musculares pelo menos duas vezes por semana.
> 
> **6. Comparativo com o Plano:**
> 
> O plano de exercícios proposto está alinhado com as diretrizes da OMS, pois inclui tanto atividades de fortalecimento muscular (musculação) quanto atividades aeróbicas (aquecimento, resfriamento e sessões de corrida opcionais). A frequência e a intensidade dos treinos estão adequadas para o seu objetivo de hipertrofia e nível de atividade física atual. O plano atende aos requisitos mínimos de atividade física recomendados pela OMS, e a inclusão de sessões de corrida opcionais pode contribuir para benefícios adicionais à saúde cardiovascular.
> 
